In this notebook, we:
1. Plot the overall of effect of forbidding the correct answer.
2. Plot the importance of each terminal node for the dontsay effect.
3. Plot the effect of ablating multiple terminal nodes.

### Import